In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

## Stratified Sampling, don't run this in AWS

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession \
    .builder \
    .appName("process_data") \
    .getOrCreate()

data_dir = "../MLE_Project_Data"
data_file_name = "train_data.csv"
data_path = os.path.join(data_dir,data_file_name)

raw_df = spark.read.load(data_path,format="csv",delimiter="|",header=True)

# sample 
sampled_df = raw_df.stat.sampleBy("label",
                     {"0":0.03,"1":0.03},
                     seed=100)

# save data, don't
sample_file_name = "sampled_data.csv"
ouptput_dir = "./data"
sampled_df.write.format('csv').option('header',True)\
.mode('overwrite').option('sep',',')\
.save(os.path.join(ouptput_dir,sample_file_name))

KeyboardInterrupt: 

## Train Test Split

In [7]:
data_dir = "./data/sampled_data.csv/"

df = pd.DataFrame()
for file in os.listdir(data_dir):    
    if file[-3:] == "csv":
        df = pd.concat((df,pd.read_csv(os.path.join(data_dir,file))),axis=0)

In [11]:
! pwd

/home/wilson/MLE_Nanodegree_Capstone


In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:],df['label'],test_size=0.2,random_state=42,stratify=df['label'])

In [28]:
train_df = pd.concat((y_train, X_train), axis=1)
train_df.to_csv("./data/train.csv",index=False)